# Right Left up down

## Setup

In [1]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support

from frame_generator import FrameGenerator
from model import create_model

2024-01-04 21:26:32.654617: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-04 21:26:35.216368: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-04 21:26:35.216400: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-01-04 21:26:35.438495: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-04 21:26:39.129861: W tensorflow/stream_executor/platform/de

In [3]:
import cv2
import numpy as np

def get_video_lngth(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Failed to open video: {video_path}")
        return -1

    # Get the number of frames in the video
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return length


def get_video_lengths(folder_path):
    # List to store the lengths of the videos
    video_lengths = []

    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".avi"):  # Check for video files (e.g., .mp4)
            video_path = os.path.join(folder_path, filename)

            length = get_video_lngth(video_path)
            if length == -1:
                continue

            video_lengths.append(length)

            # Release the VideoCapture object
            

    return np.array(video_lengths)

lengths_array = get_video_lengths("./data_right_left_up_down_1200/test/")
np.append(lengths_array, get_video_lengths("./data_right_left_up_down_1200/train/"))
np.append(lengths_array, get_video_lengths("./data_right_left_up_down_1200/validation/"))

# Calculate average
average = np.mean(lengths_array)

# Calculate maximum
maximum = np.max(lengths_array)

# Calculate minimum
minimum = np.min(lengths_array)

print("Average:", average)
print("Maximum:", maximum)
print("Minimum:", minimum)


Average: 46.21875
Maximum: 71
Minimum: 30


In [4]:
DATA_PATH = "../data_right_left_up_down_1200"
NOTEBOOK_NAME = "2_2d_plus_1"
RESULTS_PATH = DATA_PATH + "/" + NOTEBOOK_NAME

# PARAMS
# number of frames taken from each video
n_frames = 36
# number of frames skipped from each video
frame_step = 2
batch_size = 8
# Define the dimensions of one frame in the set of frames created
HEIGHT = 224
WIDTH = 224

assert(os.path.isdir(DATA_PATH ))

if not os.path.isdir(RESULTS_PATH):
    os.mkdir(RESULTS_PATH)

index_df = pd.read_csv(f'{DATA_PATH}/indx_df.csv')

print(f"classes being compared {index_df['category'].unique()}")

classes being compared ['Pushing something from left to right'
 'Pushing something from right to left' 'Moving something up'
 'Moving something down']


## Preprocess video data

Load something something data tf.data.Dataset

In [5]:
subset_paths = {
    "test": Path(f'{DATA_PATH}/test'),
    "train": Path(f'{DATA_PATH}/train'),
    "val": Path(f'{DATA_PATH}/validation'),
}

output_signature = (
    tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
    tf.TensorSpec(shape = (), dtype = tf.int16)
)

train_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['train'],
        n_frames=n_frames,
        index_df=index_df,
        frame_step=frame_step,
        height=HEIGHT,
        width=WIDTH,
        training=True
    ),
    output_signature = output_signature
)
# Batch the data
train_ds = train_ds.batch(batch_size)

val_ds = tf.data.Dataset.from_generator(
    FrameGenerator(
        subset_paths['val'],
        n_frames=n_frames,
        index_df=index_df,
        frame_step=frame_step,
        height=HEIGHT,
        width=WIDTH,
        training=False
    ),
    output_signature = output_signature
)
# Batch the data
val_ds = val_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(
    FrameGenerator(
        subset_paths['test'],
        n_frames=n_frames,
        index_df=index_df,
        frame_step=frame_step,
        height=HEIGHT,
        width=WIDTH,
        training=False
    ),
    output_signature = output_signature
)

print(type(test_ds))

# Batch the data
test_ds = test_ds.batch(batch_size)

print(type(test_ds))


<class 'tensorflow.python.data.ops.dataset_ops.FlatMapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


2024-01-04 21:28:16.742739: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/python/3.10.13/lib/python3.10/site-packages/cv2/../../lib64:
2024-01-04 21:28:16.752256: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-01-04 21:28:16.752289: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-5f4282): /proc/driver/nvidia/version does not exist
2024-01-04 21:28:16.767203: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Model Creation

In [6]:
model = create_model(n_frames=n_frames, height=HEIGHT, width=WIDTH)

## Build Model

In [8]:
frames, label = next(iter(train_ds))

In [9]:
model.build(frames)

## Load the Model

Using BinaryCrossentropy as it is more effective for binary data

from_logits is false because final layer includes a sigmoid activation,

In [12]:
previous_runs = 4

In [13]:
previous_runs += 1

model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001),
              metrics=[
                    'accuracy',
                ]
            )

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    RESULTS_PATH + '/model-runs-' + str(previous_runs) + '-cp-{epoch:02d}-{val_loss:.2f}.ckpt',
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    save_weights_only=True
)


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [14]:
latest = tf.train.latest_checkpoint(RESULTS_PATH)

if latest is not None:
    print(f"loading model from weights: {latest}")
    model.load_weights(latest)

loading model from weights: data_right_left_up_down_1200/2_2d_plus_1_right_left_n_step_2_model_save_BinaryCrossentropy_precision_recall/model-runs-4-cp-12-0.24.ckpt


## Train the Model

In [15]:
previously_run_epochs = 8 + 11 + 12 + 12

history = model.fit(
        x=train_ds,
        epochs = 50 - previously_run_epochs,
        validation_data=val_ds,
        callbacks=[checkpoint_callback, early_stopping],
    )

Epoch 1/7


2023-12-19 15:49:26.667879: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 173408256 exceeds 10% of free system memory.


      1/Unknown - 18s 18s/step - loss: 0.1191 - accuracy: 1.0000

2023-12-19 15:49:37.350841: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 173408256 exceeds 10% of free system memory.


      2/Unknown - 22s 4s/step - loss: 0.0873 - accuracy: 1.0000 

2023-12-19 15:49:41.376517: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 173408256 exceeds 10% of free system memory.


      3/Unknown - 26s 4s/step - loss: 0.1435 - accuracy: 0.9583

2023-12-19 15:49:44.775021: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 173408256 exceeds 10% of free system memory.


480/480 [==============================] - 1690s 3s/step - loss: 0.1505 - accuracy: 0.9508 - val_loss: 0.2486 - val_accuracy: 0.9312
Epoch 2/7
480/480 [==============================] - 1675s 3s/step - loss: 0.1460 - accuracy: 0.9531 - val_loss: 0.2695 - val_accuracy: 0.9042
Epoch 3/7
480/480 [==============================] - 1662s 3s/step - loss: 0.1503 - accuracy: 0.9500 - val_loss: 0.2463 - val_accuracy: 0.9125
Epoch 4/7
480/480 [==============================] - 1663s 3s/step - loss: 0.1460 - accuracy: 0.9497 - val_loss: 0.3020 - val_accuracy: 0.9021
Epoch 5/7
480/480 [==============================] - 1647s 3s/step - loss: 0.1200 - accuracy: 0.9612 - val_loss: 0.3297 - val_accuracy: 0.8854
Epoch 6/7
480/480 [==============================] - 1751s 4s/step - loss: 0.1443 - accuracy: 0.9560 - val_loss: 0.3756 - val_accuracy: 0.8583
Epoch 7/7
480/480 [==============================] - 1781s 4s/step - loss: 0.1237 - accuracy: 0.9599 - val_loss: 0.2939 - val_accuracy: 0.8917


## Metrics

### Accuracy, average precision,  average recall

In [14]:
## In future should have all 4 metrics but now just has loss and accuracy
loss, accuracy, precision, recall = model.evaluate(test_ds)

print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")
print(f"average Precision: {precision}")
print(f"average Recall: {recall}")

2023-12-07 17:53:38.140754: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1104674816 exceeds 10% of free system memory.
2023-12-07 17:53:40.973318: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1104674816 exceeds 10% of free system memory.
2023-12-07 17:53:41.878746: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1104674816 exceeds 10% of free system memory.


      1/Unknown - 10s 10s/step - loss: 0.6355 - accuracy: 0.6250 - precision: 0.6667 - recall: 0.5000

2023-12-07 17:53:46.245460: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1104674816 exceeds 10% of free system memory.
2023-12-07 17:53:49.054489: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1104674816 exceeds 10% of free system memory.


30/30 [==============================] - 264s 9s/step - loss: 0.6022 - accuracy: 0.7000 - precision: 0.7400 - recall: 0.6167
Loss: 0.6021633744239807
Accuracy: 0.699999988079071
average Precision: 0.7400000095367432
average Recall: 0.6166666746139526


## Analyse results

In [ ]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

true_labels = []
predictions = []

for batch in test_ds:
    x, y = batch
    true_labels.extend(y.numpy())
    preds = model.predict(x)
    preds = softmax(preds, axis=1)
    preds = np.argmax(preds, axis=1)
    predictions.extend(preds)

true_labels = np.array(true_labels)
predictions = np.array(predictions)

In [ ]:
fg = FrameGenerator(
    subset_paths['test'],
    n_frames=n_frames,
    index_df=index_df,
    frame_step=frame_step,
    training=False
)
class_id_value = {
    fg.class_ids_for_name[x]: x for x in fg.class_ids_for_name.keys()
 }

true_labels = np.array(true_labels)
predictions = np.array(predictions)

accuracy = accuracy_score(true_labels, predictions)

precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predictions, average=None)

print(f"Overall Accuracy: {accuracy}")
for i, (prec, rec, f1) in enumerate(zip(precision, recall, f1_score)):
    print(f"Class {class_id_value[i]}: Precision: {prec}, Recall: {rec}, F1 Score: {f1}")